In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [43]:
dataset = pd.read_csv("heart.csv")
dataset.head()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [44]:
dataset.dtypes


age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object

In [45]:
X = dataset.iloc[:, :-1]
X.head()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2


In [46]:
# print(X["restecg"].unique())
# print(X["thal"].unique())
# print(X["slope"].unique())
# print(X["cp"].unique())


In [47]:
y = dataset.iloc[:, [-1]]
y.head()


,target
0,1
1,1
2,1
3,1
4,1


In [48]:
X_ohe = pd.get_dummies(X, columns=["cp", "restecg", "slope", "thal"])


In [49]:
X_ohe.columns


Index(['age', 'sex', 'trestbps', 'chol', 'fbs', 'thalach', 'exang', 'oldpeak',
       'ca', 'cp_0', 'cp_1', 'cp_2', 'cp_3', 'restecg_0', 'restecg_1',
       'restecg_2', 'slope_0', 'slope_1', 'slope_2', 'thal_0', 'thal_1',
       'thal_2', 'thal_3'],
      dtype='object')

In [50]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_ohe, y, test_size=0.2, random_state=0
)


In [51]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


In [52]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score


In [53]:
def build_classifier():
  classifier = Sequential()
  classifier.add(keras.Input(shape=(23,)))
  classifier.add(Dense(11, activation="relu"))
  classifier.add(Dropout(0.1))
  classifier.add(Dense(11, activation="relu"))
  classifier.add(Dropout(0.1))
  classifier.add(Dense(1, activation="sigmoid"))
  classifier.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
  return classifier


In [54]:
classifier = KerasClassifier(build_fn=build_classifier, batch_size=10, epochs=100)
accuracies = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10, n_jobs=-1)
mean = accuracies.mean()
variance = accuracies.std()
print(mean)
print(variance)

/var/folders/9t/1591ynh12mb11ddqfryb4ck00000gn/T/ipykernel_5970/3955631495.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn=build_classifier, batch_size=10, epochs=100)


Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
22/22 [==============================] - 1s 4ms/step - loss: 0.6942 - accuracy: 0.5734
Epoch 2/100
22/22 [==============================] - 1s 5ms/step - loss: 0.8130 - accuracy: 0.4541
Epoch 2/100
 1/22 [>.............................] - ETA: 0s - loss: 0.7922 - accuracy: 0.6000Epoch 2/100
Epoch 2/100
22/22 [==============================] - 1s 4ms/step - loss: 0.7863 - accuracy: 0.5459
Epoch 2/100
Epoch 2/100
22/22 [==============================] - 0s 3ms/step - loss: 0.6155 - accuracy: 0.6743
Epoch 3/100
22/22 [==============================] - 0s 3ms/step - loss: 0.6682 - accuracy: 0.5760
Epoch 3/100
Epoch 3/100
17/22 [======================>.......] - ETA: 0s - loss: 0.6273 - accuracy: 0.7235Epoch 3/100
Epoch 3/100
22/22 [==============================] - 0s 4ms/step - loss: 0.6211 - accuracy: 0.7064
Epoch 3/100
22/22 [==============================] - 0s 4ms/step - loss: 0.5989 - accu

3/3 [==============================] - 0s 2ms/step - loss: 0.6144 - accuracy: 0.7083
Epoch 1/100
Epoch 1/100
22/22 [==============================] - 1s 8ms/step - loss: 0.7032 - accuracy: 0.6147
Epoch 2/100
22/22 [==============================] - 1s 7ms/step - loss: 0.6142 - accuracy: 0.6514
Epoch 2/100
22/22 [==============================] - 0s 2ms/step - loss: 0.6046 - accuracy: 0.7248
Epoch 3/100
22/22 [==============================] - 0s 2ms/step - loss: 0.5843 - accuracy: 0.6789
Epoch 3/100
22/22 [==============================] - 0s 2ms/step - loss: 0.5604 - accuracy: 0.7431
Epoch 4/100
22/22 [==============================] - 0s 1ms/step - loss: 0.5566 - accuracy: 0.7156
Epoch 4/100
22/22 [==============================] - 0s 1ms/step - loss: 0.5188 - accuracy: 0.7890
Epoch 5/100
22/22 [==============================] - 0s 2ms/step - loss: 0.5227 - accuracy: 0.7385
Epoch 5/100
22/22 [==============================] - 0s 2ms/step - loss: 0.4720 - accuracy: 0.8119
Epoch 6/100


In [55]:
classifier.fit(X_train, y_train, batch_size=10, epochs=100)


Epoch 1/100
25/25 [==============================] - 0s 1ms/step - loss: 0.7267 - accuracy: 0.4132
Epoch 2/100
25/25 [==============================] - 0s 1ms/step - loss: 0.6899 - accuracy: 0.5331
Epoch 3/100
25/25 [==============================] - 0s 1ms/step - loss: 0.6558 - accuracy: 0.6240
Epoch 4/100
25/25 [==============================] - 0s 2ms/step - loss: 0.6116 - accuracy: 0.6983
Epoch 5/100
25/25 [==============================] - 0s 2ms/step - loss: 0.5906 - accuracy: 0.7397
Epoch 6/100
25/25 [==============================] - 0s 2ms/step - loss: 0.5701 - accuracy: 0.7479
Epoch 7/100
25/25 [==============================] - 0s 2ms/step - loss: 0.5335 - accuracy: 0.7934
Epoch 8/100
25/25 [==============================] - 0s 2ms/step - loss: 0.5056 - accuracy: 0.7893
Epoch 9/100
25/25 [==============================] - 0s 2ms/step - loss: 0.4564 - accuracy: 0.8058
Epoch 10/100
25/25 [==============================] - 0s 1ms/step - loss: 0.4503 - accuracy: 0.8182
Epoch 11/

In [56]:

y_pred = classifier.predict(X_test)


In [57]:
y_pred = y_pred > 0.5


In [58]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)


In [59]:
cm


array([[23,  4],
       [ 2, 32]])

In [62]:
acc = accuracy_score(y_test, y_pred) * 100


In [63]:
acc


90.1639344262295